# eye detection

# eye detection using mediapipe face detection model and face mesh model

In [1]:
import cv2
import mediapipe as mp

2024-02-29 15:38:28.794438: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-29 15:38:29.025556: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-29 15:38:29.025600: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-29 15:38:29.046659: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-29 15:38:29.071947: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-29 15:38:29.073714: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [12]:
import shutil

def copy_and_append_new_bounding_boxes(original_file_path, new_file_path, all_bboxes):
    """
    Copies an existing annotation file and appends new bounding boxes with class labels to the copy.
    
    Parameters:
    - original_file_path: Path to the original annotation file.
    - new_file_path: Path for the new annotation file where the content is to be copied and new bounding boxes appended.
    - new_bboxes: List of new bounding boxes to append, where each bounding box is 
                  represented as (class_label, x_center_norm, y_center_norm, width_norm, height_norm).
    """
    # Copy the original file to a new location
    shutil.copy(original_file_path, new_file_path)
    
    # Append new bounding boxes to the copied file
    with open(new_file_path, 'a') as file:  # Open file in append mode
        for new_bboxes in all_bboxes:
            for bbox in new_bboxes:
                line = ' '.join(map(str, bbox)) + '\n'  # Convert each bbox to a string line
                file.write(line)  # Append the new line to the file

In [2]:
# Function to calculate the bounding box from landmarks
def calculate_bounding_box(landmarks, image_width, image_height):
    x_coordinates = [landmark.x for landmark in landmarks]
    y_coordinates = [landmark.y for landmark in landmarks]
    
    xmin = min(x_coordinates) * image_width
    xmax = max(x_coordinates) * image_width
    ymin = min(y_coordinates) * image_height
    ymax = max(y_coordinates) * image_height
    
    return int(xmin), int(ymin), int(xmax), int(ymax)

In [3]:
def convert_to_yolo_format(xmin, ymin, xmax, ymax, image_width, image_height):
    x_center = (xmin + xmax) / 2
    y_center = (ymin + ymax) / 2
    width = xmax - xmin
    height = ymax - ymin
    
    # Normalize
    x_center_norm = x_center / image_width
    y_center_norm = y_center / image_height
    width_norm = width / image_width
    height_norm = height / image_height
    
    return (x_center_norm, y_center_norm, width_norm, height_norm)

# Function to convert eye bounding boxes to YOLO format
def convert_eyes_to_yolo(left_eye_bbox, right_eye_bbox, image_width, image_height):
    left_eye_yolo = convert_to_yolo_format(*left_eye_bbox, image_width, image_height)
    right_eye_yolo = convert_to_yolo_format(*right_eye_bbox, image_width, image_height)
    return left_eye_yolo, right_eye_yolo


In [4]:
# Initialize MediaPipe Face Detection and Face Mesh.
mp_face_detection = mp.solutions.face_detection
mp_face_mesh = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils
face_detection = mp_face_detection.FaceDetection(model_selection =1, min_detection_confidence=0.5)
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=True, max_num_faces=100, refine_landmarks=True, min_detection_confidence=0.5)

I0000 00:00:1709203114.187101   28021 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1709203114.225161   28086 gl_context.cc:344] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 440.33.01), renderer: GeForce GTX 1050 Ti/PCIe/SSE2
I0000 00:00:1709203114.233384   28021 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1709203114.254774   28099 gl_context.cc:344] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 440.33.01), renderer: GeForce GTX 1050 Ti/PCIe/SSE2
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [8]:
right_eye_indices = [33, 133, 160, 158, 153, 144, 145, 153]
left_eye_indices = [362, 384, 381, 382, 263, 373, 374, 380]

In [13]:
# Load an image.
image_path = '' # add your image path
image = cv2.imread(image_path)
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

In [14]:
# Detect faces in the image.
detections = face_detection.process(image_rgb)

In [15]:
if detections.detections:
    for detection in detections.detections:
        bboxC = detection.location_data.relative_bounding_box
        ih, iw, _ = image.shape
        x, y, w, h = int(bboxC.xmin * iw), int(bboxC.ymin * ih), int(bboxC.width * iw), int(bboxC.height * ih)
        
        padding = 0.1 # You can change the padding value to whatever suits your needs
        x_padding = int(w * padding)
        y_padding = int(h * padding)

        # Modify x, y, w, and h to include padding, ensuring we don't go out of the image bounds
        x = max(0, x - x_padding)
        y = max(0, y - y_padding)
        w = min(iw - x, w + 2 * x_padding)
        h = min(ih - y, h + 2 * y_padding)

        # Apply face mesh on the enlarged detected face region.
        face_roi = image_rgb[y:y+h, x:x+w]
        
        # Apply face mesh on the detected face region.
        face_roi = image[y:y+h, x:x+w]
        
        roi_h, roi_w, _ = face_roi.shape
        mesh_results = face_mesh.process(cv2.cvtColor(face_roi, cv2.COLOR_RGB2BGR))

        if mesh_results.multi_face_landmarks:
            for face_landmarks in mesh_results.multi_face_landmarks:
                left_eye = [face_landmarks.landmark[i] for i in left_eye_indices]  # Left eye horizontal boundary.
                right_eye = [face_landmarks.landmark[i] for i in right_eye_indices]  # Right eye horizontal boundary.
                
                # Calculate bounding boxes within the face ROI.
                left_eye_bbox = calculate_bounding_box(left_eye, roi_w, roi_h)
                right_eye_bbox = calculate_bounding_box(right_eye, roi_w, roi_h)
                # Offset the bounding box coordinates to the original image's coordinate space.
                left_eye_bbox = (left_eye_bbox[0] + x, left_eye_bbox[1] + y, left_eye_bbox[2] + x, left_eye_bbox[3] + y)
                right_eye_bbox = (right_eye_bbox[0] + x, right_eye_bbox[1] + y, right_eye_bbox[2] + x, right_eye_bbox[3] + y)

                
                # Convert and print the bounding boxes in YOLO format
                left_eye_yolo, right_eye_yolo = convert_eyes_to_yolo(left_eye_bbox, right_eye_bbox, iw, ih)
                # Add class labels to the tuples
                right_eye_with_class = (2,) + right_eye_yolo  # Class 2 for right eye
                left_eye_with_class = (2,) + left_eye_yolo  # Class 2 for left eye

                # List of new bounding boxes to append
                new_bboxes = [right_eye_with_class, left_eye_with_class]
                # print(left_eye_bbox, left_eye_with_class)
                # print(right_eye_bbox, right_eye_with_class)
                
                # Call the function to copy the original file and append new bounding boxes
                # copy_and_append_new_bounding_boxes(label_path, new_file_path, new_bboxes)
                cv2.rectangle(image, (int(left_eye_bbox[0]), int(left_eye_bbox[1])), (int(left_eye_bbox[2]), int(left_eye_bbox[3])), (0, 255, 0), 2)
                cv2.rectangle(image, (int(right_eye_bbox[0]), int(right_eye_bbox[1])), (int(right_eye_bbox[2]), int(right_eye_bbox[3])), (0, 255, 0), 2)

cv2.imshow('Eyes Bounding Boxes', image)
cv2.waitKey(0)
cv2.destroyAllWindows()